# 1. Introduction

Defining clusters is important to take actions in many fields. One classic example is defining cluster of customers to sell products and services based on characteristics of clusters, making the selling process more natural.
In this project the main problem dwells in a moment of public calamity in many countries are facing – the COVID-19 pandemic. The approach is to cluster cities of São Paulo based on metrics that are released daily by Fundação SEADE – Fundação Sistema Estadual de Análise de Dados Estatísticos [1], that is a foundation related to São Paulo’s Government. 
This project will make use of this data, besides foursquare data, to cluster the cities.


# 2. Dataset

Not all columns will be used from SEADE Foundation, just a few described below:

•	nome_munic: name of the city 

•	dia: epidemiologic day

•	mes: epidemiologic month

•	obitos: deaths caused by coronavirus

•	datahora: complete date

•	casos: coronavirus cases

•	pop_60: population with age greater or equal 60 years

•	pop: total population

•	area: total area

•	latitude

•	longitude

Some columns will have their name changed to English to make the project more accessible:

•	nome_munic: name

•	dia: day

•	mes: month

•	obitos: deaths

•	datahora: date

•	casos: cases


## 2.1. Foursquare

From Fourquare® API, information about hospitals will be retrieved. To select only hospitals, the category id should be equal to 4bf58dd8d48988d196941735.

# 3. Methodology

In [1]:
#importing

from os import path
from pykml import parser
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans


import folium # map rendering library

import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
url      = "https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv"

covid_sp = pd.read_csv(url,encoding='UTF-8', sep=';')

covid_sp.head()

,nome_munic,codigo_ibge,dia,mes,datahora,casos,casos_novos,casos_pc,casos_mm7d,obitos,...,nome_drs,cod_drs,pop,pop_60,area,map_leg,map_leg_s,latitude,longitude,semana_epidem
0,Adamantina,3500105,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,Marília,5,33894,7398,41199,0,8.0,"-21,6820","-51,0737",9
1,Adolfo,3500204,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,São José do Rio Preto,15,3447,761,21106,0,8.0,"-21,2325","-49,6451",9
2,Aguaí,3500303,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,São João da Boa Vista,14,35608,5245,47455,0,8.0,"-22,0572","-46,9735",9
3,Águas da Prata,3500402,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,São João da Boa Vista,14,7797,1729,14267,0,8.0,"-21,9319","-46,7176",9
4,Águas de Lindóia,3500501,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,Campinas,3,18374,3275,6013,0,8.0,"-22,4733","-46,6314",9


In [3]:
#selecting columns that will be used for modeling and renaming columns to English

covid_sp = covid_sp[['nome_munic','dia','mes','obitos','datahora','casos','pop_60','pop','area','latitude','longitude']]

covid_sp.rename(columns={"nome_munic": "name", 
                         "dia": "day",
                         "mes": "month",
                         "obitos": "deaths",
                         "datahora": "date",
                         "casos": "cases"
                        },inplace=True)


covid_sp.head()

,name,day,month,deaths,date,cases,pop_60,pop,area,latitude,longitude
0,Adamantina,25,2,0,2020-02-25,0,7398,33894,41199,"-21,6820","-51,0737"
1,Adolfo,25,2,0,2020-02-25,0,761,3447,21106,"-21,2325","-49,6451"
2,Aguaí,25,2,0,2020-02-25,0,5245,35608,47455,"-22,0572","-46,9735"
3,Águas da Prata,25,2,0,2020-02-25,0,1729,7797,14267,"-21,9319","-46,7176"
4,Águas de Lindóia,25,2,0,2020-02-25,0,3275,18374,6013,"-22,4733","-46,6314"


In [4]:
#selecting cities with at least 1 case

covid_sp_cases = covid_sp.loc[covid_sp.cases > 0].copy()

#converting date string to date
covid_sp_cases['date'] = pd.to_datetime(covid_sp_cases['date'])

#counting numbers of days since first in
covid_sp_cases = covid_sp_cases.sort_values(by=['name','date'])
covid_sp_cases['date_diff'] = covid_sp_cases.groupby('name')['date'].diff()/ np.timedelta64(1, 'D')
covid_sp_cases['number_days'] = covid_sp_cases.groupby('name')['date_diff'].cumsum()
covid_sp_cases.head()

,name,day,month,deaths,date,cases,pop_60,pop,area,latitude,longitude,date_diff,number_days
28861,Adamantina,14,4,0,2020-04-14,1,7398,33894,41199,"-21,6820","-51,0737",NaN,NaN
29450,Adamantina,15,4,0,2020-04-15,1,7398,33894,41199,"-21,6820","-51,0737",1.0,1.0
30039,Adamantina,16,4,0,2020-04-16,1,7398,33894,41199,"-21,6820","-51,0737",1.0,2.0
30628,Adamantina,17,4,0,2020-04-17,1,7398,33894,41199,"-21,6820","-51,0737",1.0,3.0
31217,Adamantina,18,4,0,2020-04-18,1,7398,33894,41199,"-21,6820","-51,0737",1.0,4.0


In [5]:
covid_sp_cases.head()

,name,day,month,deaths,date,cases,pop_60,pop,area,latitude,longitude,date_diff,number_days
28861,Adamantina,14,4,0,2020-04-14,1,7398,33894,41199,"-21,6820","-51,0737",NaN,NaN
29450,Adamantina,15,4,0,2020-04-15,1,7398,33894,41199,"-21,6820","-51,0737",1.0,1.0
30039,Adamantina,16,4,0,2020-04-16,1,7398,33894,41199,"-21,6820","-51,0737",1.0,2.0
30628,Adamantina,17,4,0,2020-04-17,1,7398,33894,41199,"-21,6820","-51,0737",1.0,3.0
31217,Adamantina,18,4,0,2020-04-18,1,7398,33894,41199,"-21,6820","-51,0737",1.0,4.0


In [6]:
#selecting data until 60th day of coronavirus infection
covid_sp_cases_days = covid_sp_cases[covid_sp_cases['number_days']==60.0]

#selecting only few columns
covid_sp_cases_days = covid_sp_cases_days[['name','deaths','cases','pop_60','pop','area','latitude','longitude', 'number_days']].reset_index(drop=True)

#creating column of number of cases per 100M habitants
covid_sp_cases_days['cases_100M'] = covid_sp_cases_days['cases']/covid_sp_cases_days['pop']*100000

#creating column of IFR
covid_sp_cases_days['IFR'] = covid_sp_cases_days['deaths']/covid_sp_cases_days['cases']

#creating column of number of deaths per 100M habitants
covid_sp_cases_days['deaths_100M'] = covid_sp_cases_days['deaths']/covid_sp_cases_days['pop']*100000

In [7]:
#formatting latitude and longitude as numbers of type float
covid_sp_cases_days['latitude'] = covid_sp_cases_days['latitude'].str.replace(',','.').astype(float)
covid_sp_cases_days['longitude'] = covid_sp_cases_days['longitude'].str.replace(',','.').astype(float)

#deleting ignored cases in dataset from SEADE
covid_sp_cases_days = covid_sp_cases_days[covid_sp_cases_days['name']!="Ignorado"]

#saving to excel
covid_sp_cases_days.to_excel("covid_sp_cases_days.xlsx")  

In [8]:
covid_sp_cases_days.head()

,name,deaths,cases,pop_60,pop,area,latitude,longitude,number_days,cases_100M,IFR,deaths_100M
0,Adamantina,3,44,7398,33894,41199,-21.6820,-51.0737,60.0,129.816487,0.068182,8.851124
1,Agudos,1,23,5524,36134,96671,-22.4694,-48.9863,60.0,63.651962,0.043478,2.767477
2,Americana,6,118,40276,233458,13391,-22.7374,-47.3331,60.0,50.544423,0.050847,2.570055
3,Amparo,5,83,12727,69639,44532,-22.7088,-46.7720,60.0,119.186088,0.060241,7.179885
4,Américo Brasiliense,2,58,4465,40243,12279,-21.7288,-48.1147,60.0,144.124444,0.034483,4.969808


In [9]:
covid_sp_cases_days.dtypes

name            object
deaths           int64
cases            int64
pop_60           int64
pop              int64
area             int64
latitude       float64
longitude      float64
number_days    float64
cases_100M     float64
IFR            float64
deaths_100M    float64
dtype: object

In [ ]:
CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [37]:
def getNearbyHospitals(names, latitudes, longitudes, areas, LIMIT=100):
    
    column_names = ['name', 
                  'number_hospitals']
    
    nearby_hospitals = pd.DataFrame(columns = column_names)
    
    for name, lat, lng, area in zip(names, latitudes, longitudes, areas):
           
        radius = np.sqrt(area * 10000/np.pi)
        
        #Hospital category ID
        categoryId = '4bf58dd8d48988d196941735'
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            categoryId,
            radius, 
            LIMIT)
            
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        new_row = {'name':name, 
                    'number_hospitals':len(results)}
        nearby_hospitals = nearby_hospitals.append(new_row, ignore_index=True)
    
    return(nearby_hospitals)


nearby_hospitals= getNearbyHospitals(names=covid_sp_cases_days['name'],
                                   latitudes=covid_sp_cases_days['latitude'],
                                   longitudes=covid_sp_cases_days['longitude'],
                                    areas = covid_sp_cases_days['area'])
nearby_hospitals

KeyError: 'groups'

In [15]:
#merging about hospitals to original dataset
covid_sp_cases_days_hosp = pd.merge(covid_sp_cases_days,nearby_hospitals,left_on='name',right_on='name',how='left')

In [16]:
covid_sp_cases_days_hosp.head()

,name,deaths,cases,pop_60,pop,area,latitude,longitude,number_days,cases_100M,IFR,deaths_100M,number_hospitals
0,Adamantina,3,44,7398,33894,41199,-21.6820,-51.0737,60.0,129.816487,0.068182,8.851124,4
1,Agudos,1,23,5524,36134,96671,-22.4694,-48.9863,60.0,63.651962,0.043478,2.767477,13
2,Americana,6,118,40276,233458,13391,-22.7374,-47.3331,60.0,50.544423,0.050847,2.570055,15
3,Amparo,5,83,12727,69639,44532,-22.7088,-46.7720,60.0,119.186088,0.060241,7.179885,3
4,Américo Brasiliense,2,58,4465,40243,12279,-21.7288,-48.1147,60.0,144.124444,0.034483,4.969808,5


In [17]:
#saving to excel
covid_sp_cases_days_hosp.to_excel("covid_sp_cases_60_day.xlsx")

In [18]:
#creating column of rate of people with 60 years or more
covid_sp_cases_days_hosp['pop_60_rate'] = covid_sp_cases_days_hosp['pop_60']/covid_sp_cases_days_hosp['pop']

#creating column of number of hospitals per 100M habitants
covid_sp_cases_days_hosp['number_hospitals_100M'] = covid_sp_cases_days_hosp['number_hospitals'].astype(float)/covid_sp_cases_days_hosp['pop']*100000
covid_sp_cases_days_hosp.head()

,name,deaths,cases,pop_60,pop,area,latitude,longitude,number_days,cases_100M,IFR,deaths_100M,number_hospitals,pop_60_rate,number_hospitals_100M
0,Adamantina,3,44,7398,33894,41199,-21.6820,-51.0737,60.0,129.816487,0.068182,8.851124,4,0.218269,11.801499
1,Agudos,1,23,5524,36134,96671,-22.4694,-48.9863,60.0,63.651962,0.043478,2.767477,13,0.152875,35.977196
2,Americana,6,118,40276,233458,13391,-22.7374,-47.3331,60.0,50.544423,0.050847,2.570055,15,0.172519,6.425139
3,Amparo,5,83,12727,69639,44532,-22.7088,-46.7720,60.0,119.186088,0.060241,7.179885,3,0.182757,4.307931
4,Américo Brasiliense,2,58,4465,40243,12279,-21.7288,-48.1147,60.0,144.124444,0.034483,4.969808,5,0.110951,12.424521


In [19]:
#columns that will be used by K-Means Algorithm
mod_columns = ['IFR','cases_100M', 'number_hospitals_100M', 'pop_60_rate','deaths_100M']
covid_sp_cases_days_clustering = covid_sp_cases_days_hosp[mod_columns]
covid_sp_cases_days_clustering.dtypes

IFR                      float64
cases_100M               float64
number_hospitals_100M    float64
pop_60_rate              float64
deaths_100M              float64
dtype: object

In [20]:
#Verifying missing values
covid_sp_cases_days_clustering.columns[covid_sp_cases_days_clustering.isnull().any()]

Index([], dtype='object')

In [21]:
from sklearn.preprocessing import StandardScaler

#applying StandardScaler
X = covid_sp_cases_days_clustering.values
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)

X

array([[6.81818182e-02, 1.29816487e+02, 1.18014988e+01, 2.18268720e-01,
        8.85112409e+00],
       [4.34782609e-02, 6.36519621e+01, 3.59771960e+01, 1.52875408e-01,
        2.76747661e+00],
       [5.08474576e-02, 5.05444234e+01, 6.42513857e+00, 1.72519254e-01,
        2.57005543e+00],
       [6.02409639e-02, 1.19186088e+02, 4.30793090e+00, 1.82756789e-01,
        7.17988483e+00],
       [3.44827586e-02, 1.44124444e+02, 1.24245210e+01, 1.10950973e-01,
        4.96980841e+00],
       [1.13636364e-01, 7.84957363e+01, 7.13597602e+00, 1.92243194e-01,
        8.91997003e+00],
       [0.00000000e+00, 1.21624909e+01, 0.00000000e+00, 1.49395930e-01,
        0.00000000e+00],
       [1.70940171e-02, 1.02803820e+02, 3.07532796e+00, 1.76558972e-01,
        1.75733026e+00],
       [2.05479452e-02, 1.11401909e+02, 3.81513387e+00, 1.64768002e-01,
        2.28908032e+00],
       [4.16666667e-02, 1.14394662e+02, 0.00000000e+00, 1.04528122e-01,
        4.76644423e+00],
       [3.10559006e-02, 8.4528

In [32]:
from sklearn.metrics import silhouette_score

sil = []
kmax = 20
max_score = 0
k_max_score = 0

#selecting best k through the max Silhouette Score
for k in range(2, kmax+1):
    kmeans =  KMeans(init="k-means++", n_clusters=k, n_init=1000, random_state=14).fit(cluster_dataset)
    labels = kmeans.labels_
    
    sil_score = silhouette_score(cluster_dataset, labels, metric = 'euclidean')
    print(k,sil_score)
    if sil_score > max_score:
        max_score = sil_score
        k_max_score = k
    
num_clusters = k_max_score

print(num_clusters)
k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=1000, random_state=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

2 0.3371193243648176
3 0.3175849125234627
4 0.3406515340861057
5 0.33463780543121197
6 0.2589811115149869
7 0.2716510341101984
8 0.23341051524330036
9 0.21816722679023642
10 0.2186855281369933
11 0.2192304660780765
12 0.22984396404858598
13 0.22011554176353632
14 0.2303102993213655
15 0.2278057680851709
16 0.22344120547101692
17 0.22957129924300038
18 0.23795879840973874
19 0.2176204218407263
20 0.23349352952589453
4


# 4. Results and Discussion

In [33]:
#aggregating clusters information

covid_sp_cases_days_hosp.drop(['Cluster Labels'], axis=1, inplace=True,  errors='ignore')
covid_sp_cases_days_hosp.insert(0, 'Cluster Labels', labels)

In [34]:
covid_sp_cases_days_hosp.head()

,Cluster Labels,name,deaths,cases,pop_60,pop,area,latitude,longitude,number_days,cases_100M,IFR,deaths_100M,number_hospitals,pop_60_rate,number_hospitals_100M
0,1,Adamantina,3,44,7398,33894,41199,-21.6820,-51.0737,60.0,129.816487,0.068182,8.851124,4,0.218269,11.801499
1,2,Agudos,1,23,5524,36134,96671,-22.4694,-48.9863,60.0,63.651962,0.043478,2.767477,13,0.152875,35.977196
2,1,Americana,6,118,40276,233458,13391,-22.7374,-47.3331,60.0,50.544423,0.050847,2.570055,15,0.172519,6.425139
3,1,Amparo,5,83,12727,69639,44532,-22.7088,-46.7720,60.0,119.186088,0.060241,7.179885,3,0.182757,4.307931
4,1,Américo Brasiliense,2,58,4465,40243,12279,-21.7288,-48.1147,60.0,144.124444,0.034483,4.969808,5,0.110951,12.424521


In [35]:
latitude = -23.5505199
longitude = -46.63330939999997

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(covid_sp_cases_days_hosp['latitude'], covid_sp_cases_days_hosp['longitude'], covid_sp_cases_days_hosp['name'], covid_sp_cases_days_hosp['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
#showing information about cluesters
for k in range(0, num_clusters):
    print(covid_sp_cases_days_hosp.loc[covid_sp_cases_days_hosp['Cluster Labels'] == k, mod_columns].describe().loc[['mean']])

           IFR  cases_100M  number_hospitals_100M  pop_60_rate  deaths_100M
mean  0.104876  141.977908               3.725253     0.140039    13.383871
           IFR  cases_100M  number_hospitals_100M  pop_60_rate  deaths_100M
mean  0.035716   98.502045               5.403868     0.163691     3.481913
           IFR  cases_100M  number_hospitals_100M  pop_60_rate  deaths_100M
mean  0.039642  131.207272              37.288564     0.161087     4.912472
           IFR  cases_100M  number_hospitals_100M  pop_60_rate  deaths_100M
mean  0.032114  734.979526              15.171242     0.166534    20.511717


Cluster 0 - Low number of Hospitals, High Deaths per 100M, High IFR

Cluster 1 - Low IFR, Low deaths per 100M, high population with 60 years or more

Cluster 2 - High Number of Hospitals, High Pop with 60 years or more, Low Deaths per 100M

Clueter 3 - High Number of Hospitals, High Deaths per 100M, High Pop with 60 years or more, Low IFR



# 5. Conclusion

Some cluster have been obtained from features like IFR, number of cases per 100M habitants, number of hospitals per 100M habitants, rate of population with 60 years or more and deaths per 100M. One action that could be taken based on cluster was to create more hospitals in cities from cluster 0 that has low numbers of hospitals and high number of deaths per 100M habitants and high IFR. One important thing to note in this analysis is that foursquare only gives 100 venues from a search, which could make hard to distinguish cities with more than 100 hospitals.

# 6. References

[1] SEADE. Github data repository. https://github.com/seade-R/dados-covid-sp

[2] Medium. How to Determine the Optimal K for K-Means. https://medium.com/analytics-vidhya/how-to-determine-the-optimal-k-for-k-means-708505d204eb
